In [1]:
from pyspark.sql.functions import col, sum, avg, count, max, min, broadcast, unix_timestamp, hour, dayofweek, format_string, when, dayofweek
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.functions import lit
from datetime import date

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5,application_1753198312670_0006,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession.builder.appName("homework3_sn").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
S3_BUCKET_NAME = "robot-dreams-source-data"
TAXI_ZONE_LOOKUP_PATH = f"s3a://{S3_BUCKET_NAME}/home-work-1/nyc_taxi/taxi_zone_lookup.csv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 2: Імпорт, уніфікація та об’єднання

#### Prepare Green taxi file

In [4]:
green_taxi_union_list = []

# prepare list of columns to cast
columns_to_cast = {
    "passenger_count": "bigint",
    "payment_type": "bigint",
    "improvement_surcharge": "double",
    "congestion_surcharge": "double",
    "PULocationID": "bigint"
}

# loop over the files to cast each file and collect them into a list
for folder in range(2014,2024):
    for file in range(1,13):
        if (folder != 2023 and file not in (10,11,12)):
            GREEN_TAXI_PATH_FILE = f"s3a://{S3_BUCKET_NAME}/home-work-1/nyc_taxi/green/{folder}/green_tripdata_{folder}-{str(file).zfill(2)}.parquet"

            df = spark.read.parquet(GREEN_TAXI_PATH_FILE)

            for col_name, target_type in columns_to_cast.items():
                df = df.withColumn(col_name, col(col_name).cast(target_type))

            green_taxi_union_list.append(df)
    
# combine all dfs from the list    
green_taxi_final = green_taxi_union_list[0]

for df in green_taxi_union_list[1:]:
    green_taxi_final = green_taxi_final.unionByName(df)

# add taxi type
green_taxi_final = green_taxi_final.withColumn("taxi_type", lit("green"))

print(f"Total rows count green taxi file: {green_taxi_final.count()}")

green_taxi_final.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total rows count green taxi file: 61956492
root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- lpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- taxi_type: string (nullable = false)

#### Prepare Yellow taxi file

In [5]:
yellow_taxi_union_list = []

# prepare columns to cast
columns_to_cast = {
    "PULocationID": "bigint"
}

# prepare columns to add
missing_columns = ['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'ehail_fee', 'trip_type']

# loop over the files to cast each file and collect them into a list
for folder in range(2014,2024):
    if folder not in (2019,2021):   # exclude missing folders
        for file in range(1,13):
            if not ((folder == 2023 and file in (10, 11, 12)) or (folder == 2020 and file == 2)):             # exclude missing files
                YELLOW_TAXI_PATH_FILE = f"s3a://{S3_BUCKET_NAME}/home-work-1/nyc_taxi/yellow/{folder}/yellow_tripdata_{folder}-{str(file).zfill(2)}.parquet"

                df = spark.read.parquet(YELLOW_TAXI_PATH_FILE)

                for col_name in missing_columns:
                    if col_name not in df.columns:
                        df = df.withColumn(col_name, lit(None))

                for col_name, target_type in columns_to_cast.items():
                    df = df.withColumn(col_name, col(col_name).cast(target_type))

                yellow_taxi_union_list.append(df)
    
### Align all colums    

# Get all column names
all_columns = set()
for df in yellow_taxi_union_list:
    all_columns.update(df.columns)
all_columns = sorted(all_columns)

# Align all DataFrames to have same columns
def align_columns(df, all_cols):
    for col_name in all_cols:
        if col_name not in df.columns:
            df = df.withColumn(col_name, lit(None))
    return df.select(*all_cols)  # Reorder columns

aligned_dfs = [align_columns(df, all_columns) for df in yellow_taxi_union_list]

# Union all dfs into one
yellow_taxi_final = yellow_taxi_union_list[0]

for df in yellow_taxi_union_list[1:]:
    yellow_taxi_final = yellow_taxi_final.unionByName(df)
    
yellow_taxi_final = yellow_taxi_final.withColumn("taxi_type", lit("yellow"))
    
print(f"Total rows count yellow taxi file: {yellow_taxi_final.count()}")

yellow_taxi_final.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total rows count yellow taxi file: 745067811
root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- lpep_pickup_datetime: void (nullable = true)
 |-- lpep_dropoff_datetime: void (nu

#### Add missing columns to Green taxi file

In [6]:
columns_to_add = ["tpep_pickup_datetime", "tpep_dropoff_datetime", "airport_fee"]
for col_name in columns_to_add:
        green_taxi_final = green_taxi_final.withColumn(col_name, lit(None))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Combine Yellow and Green taxi files

In [7]:
raw_trips_df = yellow_taxi_final.unionByName(green_taxi_final)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# show first row
raw_trips_df.limit(1).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+--------------------+---------------------+---------+---------+---------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|lpep_pickup_datetime|lpep_dropoff_datetime|ehail_fee|trip_type|taxi_type|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+--------------------+---------------------

#### Make transformations for the united file

In [9]:
raw_trips_df = raw_trips_df.withColumn("pickup_unix", unix_timestamp("tpep_pickup_datetime"))  # prepare proper format for calculation
raw_trips_df = raw_trips_df.withColumn("dropoff_unix", unix_timestamp("tpep_dropoff_datetime")) # prepare proper format for calculation
raw_trips_df = raw_trips_df.withColumn("duration_min", (col("dropoff_unix").cast("long") - col("pickup_unix").cast("long")) / 60)  ## calc duration minutes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# show first row
raw_trips_df.limit(1).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+--------------------+---------------------+---------+---------+---------+-----------+------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|lpep_pickup_datetime|lpep_dropoff_datetime|ehail_fee|trip_type|taxi_type|pickup_unix|dropoff_unix|duration_min|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------

In [11]:
# filter raw trips df
raw_trips_df_filtered = raw_trips_df.filter((raw_trips_df.trip_distance > 0.1) & (raw_trips_df.fare_amount > 2) & (raw_trips_df.duration_min > 1))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# show first row of filtered df
raw_trips_df_filtered.limit(1).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+--------------------+---------------------+---------+---------+---------+-----------+------------+-----------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|lpep_pickup_datetime|lpep_dropoff_datetime|ehail_fee|trip_type|taxi_type|pickup_unix|dropoff_unix|     duration_min|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+--

In [13]:
# count of rows
raw_trips_df_filtered.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

734844554

In [14]:
# prepare pickup hour and day of week
raw_trips_df = raw_trips_df.withColumn("pickup_hour", hour('tpep_pickup_datetime'))
raw_trips_df = raw_trips_df.withColumn("pickup_day_of_week", dayofweek('tpep_pickup_datetime'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Join the file with Zone Lookup with Broadcast Hash Join

In [15]:
# load csv zone lookup file
taxi_zone_lookup_df = spark.read.csv(TAXI_ZONE_LOOKUP_PATH, header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# schema for the csv file loaded
taxi_zone_lookup_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)

In [17]:
# rows count
taxi_zone_lookup_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

265

In [18]:
# first row
taxi_zone_lookup_df.limit(1).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+--------------+------------+
|LocationID|Borough|          Zone|service_zone|
+----------+-------+--------------+------------+
|         1|    EWR|Newark Airport|         EWR|
+----------+-------+--------------+------------+

In [19]:
# set a low autoBroadcastJoinThreshold, 1MB for small tables to broadcast.
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "1048576")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# select only required fields from zone lookup file
zone_lookup_df_selected = taxi_zone_lookup_df.select("LocationID", "Zone")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# join with zone lookup based on Pick Up location to take pick up zone
raw_trips_df_with_PUzone = raw_trips_df.join(
    broadcast(zone_lookup_df_selected),
    raw_trips_df.PULocationID == zone_lookup_df_selected.LocationID,
    "left_outer"
).drop('LocationID').withColumnRenamed("Zone", "pickup_zone")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# join with zone lookup based on Drop Off location to take drop off zone
raw_trips_df_with_allZone = raw_trips_df_with_PUzone.join(
    broadcast(zone_lookup_df_selected),
    raw_trips_df_with_PUzone.DOLocationID == zone_lookup_df_selected.LocationID,
    "left_outer"
).drop('LocationID').withColumnRenamed("Zone", "dropoff_zone")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# final file with zones schema
raw_trips_df_with_allZone.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- lpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- lpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- ehail_f

In [24]:
# show first row of raw trips df with all zones
raw_trips_df_with_allZone.limit(1).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+--------------------+---------------------+---------+---------+---------+-----------+------------+------------+-----------+------------------+--------------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|lpep_pickup_datetime|lpep_dropoff_datetime|ehail_fee|trip_type|taxi_type|pickup_unix|dropoff_unix|duration_min|pickup_hour|pickup_day_of_week|         pickup_zone|        dropoff_zone|
+--------+--------------------+---------------------+---------------+-------------+-------

#### 3: Створення та агрегація фінального датафрейму zone_summary

#### Make aggregations

In [ ]:
  * pickup_zone — зона посадки
  * total_trips — загальна кількість поїздок
  * avg_trip_distance — середня відстань поїздки
  * avg_total_amount — середній тариф поїздки
  * avg_tip_amount — середня сума чайових
  * yellow_share — частка жовтих поїздок у зоні
  * green_share — частка зелених поїздок у зоні
  * max_trip_distance — максимальна відстань поїздки
  * min_tip_amount — мінімальна сума чайових

In [25]:
zone_summary = raw_trips_df_with_allZone.groupBy("pickup_zone").agg(  
    count("*").alias("total_trips"),
    avg("trip_distance").alias("avg_trip_distance"),
    avg("total_amount").alias("avg_total_amount"),
    avg("tip_amount").alias("avg_tip_amount"),
    max("trip_distance").alias("max_trip_distance"),
    min("tip_amount").alias("min_tip_amount"),
    sum(when(col("taxi_type") == "yellow", 1).otherwise(0)).alias("yellow_trips"),
    sum(when(col("taxi_type") == "green", 1).otherwise(0)).alias("green_trips")
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# calculating yellos/green trip shares
zone_summary = zone_summary \
        .withColumn("yellow_share", (col("yellow_trips") / col("total_trips")) * 100) \
        .withColumn("green_share", (col("green_trips") / col("total_trips")) * 100) \
        .drop('yellow_trips','green_trips')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# show first 5 rows
zone_summary.limit(5).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+------------------+------------------+------------------+-----------------+--------------+------------------+-----------------+
|         pickup_zone|total_trips| avg_trip_distance|  avg_total_amount|    avg_tip_amount|max_trip_distance|min_tip_amount|      yellow_share|      green_share|
+--------------------+-----------+------------------+------------------+------------------+-----------------+--------------+------------------+-----------------+
|         Westerleigh|       1463| 7.084019138755982| 40.68898154477104| 3.313014354066984|            124.4|           0.0| 60.08202323991798|39.91797676008203|
|Charleston/Totten...|       2059| 23.35708110733366| 83.29833899951431|2.3402234094220495|            103.0|           0.0| 90.77221952404079|9.227780475959204|
|      Pelham Parkway|      85877|16.609644491540227|18.737122395984834|0.6045933136928399|        221944.04|           0.0|13.084993653714033|86.91500634628598|
|           Rego Park|     5

In [28]:
# saving df to S3 bucket
zone_summary.write \
    .mode("overwrite") \
    .parquet(f"s3a://ssnebolsinbucket/results/zone_statistic/{date.today()}/zone_statistic.parquet.")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 4. Агрегація по днях тижня та зонах

1. Підрахувати кількість поїздок за певний день тижня (наприклад, понеділок — Sunday, etc.).
2. Розрахувати частку поїздок з тарифами більше $30 за кожною зоною (high_fare_share).

In [29]:
# making the calculations - total trips and trips with total amount higher 30$, filtering this to valid days of week
zone_days_summary = raw_trips_df_with_allZone.groupBy("pickup_zone", "pickup_day_of_week").agg(  
    count("*").alias("total_trips"),
    sum(when(col("total_amount") > 30, 1).otherwise(0)).alias("high_price_trip")
    ).filter(col("pickup_day_of_week").between(1, 7))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
# naming the days of week
zone_days_summary = zone_days_summary.withColumn(
    "pickup_day_of_week",
     when(col("pickup_day_of_week") == 1, "Sunday")
    .when(col("pickup_day_of_week") == 2, "Monday")
    .when(col("pickup_day_of_week") == 3, "Tuesday")
    .when(col("pickup_day_of_week") == 4, "Wednesday")
    .when(col("pickup_day_of_week") == 5, "Thursday")
    .when(col("pickup_day_of_week") == 6, "Friday")
    .when(col("pickup_day_of_week") == 7, "Saturday")
    .otherwise("NA")
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
# caclulating high fare share
zone_days_summary = zone_days_summary.withColumn("high_fare_share", col("high_price_trip") / col("total_trips") * 100).drop("total_trips","high_price_trip")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
# show top 10 rows
zone_days_summary.limit(10).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+------------------+
|         pickup_zone|pickup_day_of_week|   high_fare_share|
+--------------------+------------------+------------------+
|Washington Height...|            Friday|21.785595475715237|
|         Old Astoria|          Saturday| 9.284985029768391|
|           Hudson Sq|          Thursday| 8.386313712565896|
|                SoHo|            Sunday| 5.993063364451522|
|       South Jamaica|          Thursday| 86.00909796959947|
|Marine Park/Mill ...|          Thursday|45.049019607843135|
|            Longwood|         Wednesday|32.360742705570296|
|     Carroll Gardens|          Thursday|17.234840196636178|
|        Bedford Park|            Monday| 30.58690744920993|
|Williamsbridge/Ol...|            Friday| 48.08080808080808|
+--------------------+------------------+------------------+

In [ ]:
# saving df to S3 bucket
zone_days_summary.write \
    .mode("overwrite") \
    .parquet(f"s3a://ssnebolsinbucket/results/zone_days_statstic/{date.today()}/zone_days_statstic.parquet.")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…